<a href="https://colab.research.google.com/github/Huanhuanhuang2022/gitlearn/blob/othertitlechanges/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import library and set cuda

In [ ]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as nn
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import copy
from torchvision import transforms
import re
import torch
from torchtext.legacy import data
from torchtext.vocab import Vectors
import spacy
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla V100-SXM2-16GB


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Merging tables


In [ ]:
nodeid2paperid=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/nodeid2paperid.csv",sep = ',', header=0)
train=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv",sep = ',', header=None)
test=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.csv",sep = ',', header=None)
text=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/text.csv",sep = ',', header=None)
text.columns=["paper id","title","abstract"]

In [ ]:
nodeid2paperid.columns=["node id","paper id"]
nodeid2paperid.shape
train.columns=["label", "node id"]
test.columns=["node id"]
test.head()
text.columns=["paper id", "title", "abstract"]

In [ ]:
train_node=train.merge(nodeid2paperid,how="inner",on="node id")
train_node_text=train_node.merge(text,how="inner",on="paper id")
train_node_text["fulltext"]=train_node_text["title"]+train_node_text["abstract"]
train_node_text=train_node_text.drop(["node id","paper id","title","abstract"],axis=1)
train_node_text

,label,fulltext
0,4,evasion attacks against machine learning at te...
1,5,how hard is computing parity with noisy commun...
2,8,a promise theory perspective on data networksN...
3,6,webvrgis based city bigdata 3d visualization a...
4,4,information theoretic authentication and secre...
...,...,...
59995,5,incentivizing users of data centers participat...
59996,16,semantic change detection with hypermapsChange...
59997,9,computing with polynomial ordinary differentia...
59998,8,on energy efficiency in wireless networks a ga...


In [ ]:
test_node=test.merge(nodeid2paperid,how="inner",on="node id")
test_node_text=test_node.merge(text,how="inner",on="paper id")
test_node_text["fulltext"]=test_node_text["title"]+test_node_text["abstract"]
test_node_text=test_node_text.drop(["node id","paper id","title","abstract"],axis=1)
test_node_text["label"]=np.array(0)
test_node_text=test_node_text.reindex(columns=["label","fulltext"])

## Standardize_text

In [ ]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n\#]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

#train.shape
train.fillna('_NA_')
train = standardize_text(train_node_text,"fulltext")
train.to_csv(r"/content/drive/My Drive/Colab Notebooks/trainfulltext.csv",index=False)


test.fillna('_NA_')
test = standardize_text(test_node_text, "fulltext")
test.to_csv(r"/content/drive/My Drive/Colab Notebooks/testfulltext.csv",index=False)

# Read tables

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
train_node_text=pd.read_csv("/content/drive/My Drive/Colab Notebooks/trainfulltext.csv")
test_node_text=pd.read_csv("/content/drive/My Drive/Colab Notebooks/testfulltext.csv")

# All Models Used for predition


## BERT


### Preprocess

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 13.9MB/s 
     |████████████████████████████████| 3.2MB 50.9MB/s 
     |████████████████████████████████| 890kB 48.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=672ef3c25a539f45c371f112cb5bf782eb46c900408558a85a40aca22c7450df
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
fulltext = train_node_text.fulltext.values
labels = train_node_text.label.values

In [ ]:
from transformers import BertTokenizer,LongformerTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=True,model_max_length=512)
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
# For every sentence...
for sent in fulltext:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True # Add '[CLS]' and '[SEP]
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

Token indices sequence length is longer than the specified maximum sequence length for this model (771 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
print('Original: ', fulltext[0])
print('Token IDs:', input_ids[0])

Original:  evasion attacks against machine learning at test timein security sensitive applications, the success of machine learning depends on a thorough vetting of their resistance to adversarial data  in one pertinent, well motivated attack scenario, an adversary may attempt to evade a deployed system at test time by carefully manipulating attack samples  in this work, we present a simple but effective gradient based approach that can be exploited to systematically assess the security of several, widely used classification algorithms against evasion attacks  following a recently proposed framework for security evaluation, we simulate attack scenarios that exhibit different risk levels for the classifier by increasing the attacker's knowledge of the system and her ability to manipulate attack samples  this gives the classifier designer a better picture of the classifier performance under evasion attacks, and allows him to perform a more informed model selection (or parameter setting) 

In [ ]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  1053


In [ ]:
from keras.preprocessing.sequence import pad_sequences
# Set the maximum sequence length. bert-based model only allow max 512 sequence length
MAX_LEN = 512
print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))
# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
print('\Done.')


Padding/truncating all sentences to 512 values...

Padding token: "[PAD]", ID: 0
\Done.


In [ ]:
# Create attention masks
attention_masks = []
# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [ ]:
labels = np.array(train_node_text["label"])

In [ ]:
from sklearn.model_selection import train_test_split
# Use 80% for training and 20% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.2)


In [ ]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 16
# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

### BERT model

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-cased", # Use the 12-layer BERT model, with an cased vocab.
    num_labels = 20,  
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.cuda()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# # Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5# change 2e-5(83)
                  #3e-4(no good), 1e-4(no good), 5e-5(no good), 3e-5(83....)2e-1(bad)
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup
# Number of training epochs (authors recommend between 2 and 4)
epochs = 2
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# stop running since it takes one day to train and evaluate(original run result in other notebook)
import random
import time
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
# Set the seed value all over the place to make this reproducible.
seed_val = 1
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []
total_t0 = time.time()
# Store the average loss after each epoch so we can plot them.
loss_values = []
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_loss = 0
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 400 batches.
        if step % 400 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()        
    
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
        
    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():        
           
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))


======== Epoch 1 / 2 ========
Training...


KeyboardInterrupt: ignored

### Prediction

In [ ]:
fulltext = test_node_text.fulltext.values

input_ids = []
# For every sentence...
for sent in fulltext:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True # Add '[CLS]' and '[SEP]'
                   )
    
    input_ids.append(encoded_sent)
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 
# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
# Set the batch size.  
batch_size = 16  
# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# model_save_name = 'bert2.pt'
# path = F"/content/gdrive/My Drive/{model_save_name}" 
# torch.save(model.state_dict(), path)

In [ ]:
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Put model in evaluation mode
model.eval()
 
predictions = []
# Predict 
for batch in prediction_dataloader:
  
  batch = tuple(t.to(device) for t in batch)

  b_input_ids, b_input_mask = batch

  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]
  logits = logits.detach().cpu().numpy()
  predictions.append(logits)
print('DONE.')

Predicting labels for 13,718 test sentences...


KeyboardInterrupt: ignored

In [ ]:
# For each input batch...
pred=[]
for i in range(len(predictions)):
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  pred_labels_i=pred_labels_i.tolist()
  pred=pred+pred_labels_i

In [ ]:
test_node_text["label"]=pred
final_result_bert=test_node_text
final_result_bert.columns=["id","label"]

In [ ]:
final_result_bert.to_csv(r"/content/drive/My Drive/Colab Notebooks/final_result_bert4.csv",index=False)

In [ ]:
model_save_name = 'bert.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model.load_state_dict(torch.load(path))

##CNN

### Preprocess


In [ ]:
# import pretrained word-embeddings
%%time
URL = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip"
FILE = "fastText"

if os.path.isdir(FILE):
    print("fastText exists.")
else:
    !wget -P $FILE $URL
    !unzip $FILE/crawl-300d-2M.vec.zip -d $FILE

fastText exists.
CPU times: user 1.67 ms, sys: 0 ns, total: 1.67 ms
Wall time: 2.02 ms


In [ ]:
from nltk.tokenize import word_tokenize
from collections import defaultdict

def tokenize(texts):
    max_len = 0
    tokenized_texts = []
    word2idx = {}

    # Add <pad> and <unk> tokens to the vocabulary
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1

    # Building vocab from the corpus starting from index 2
    idx = 2
    for sent in texts:
        tokenized_sent = word_tokenize(sent)

        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)

        # Add new token to `word2idx`
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1

        # Update `max_len`
        max_len = max(max_len, len(tokenized_sent))

    return tokenized_texts, word2idx, max_len

def encode(tokenized_texts, word2idx, max_len):
    ###Pad each sentence to the maximum sentence length and encode tokens to their index in the vocabulary.

    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in tokenized_sent]
        input_ids.append(input_id)
    
    return np.array(input_ids)#shape (N, max_len)

In [ ]:
from tqdm import tqdm_notebook
import nltk
nltk.download('punkt')

def load_pretrained_vectors(word2idx, fname):
    """Load pretrained vectors and create embedding layers.
    
    Args:
        word2idx (Dict): Vocabulary built from the corpus
        fname (str): Path to pretrained vector file

    Returns:
        embeddings (np.array): Embedding matrix with shape (N, d) where N is
            the size of word2idx and d is embedding dimension
    """

    print("Loading pretrained vectors...")
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())

    # Initilize random embeddings
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for line in tqdm_notebook(fin):
        tokens = line.rstrip().split(' ')
        word = tokens[0]
        if word in word2idx:
            count += 1
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

    print(f"There are {count} / {len(word2idx)} pretrained vectors found.")

    return embeddings

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print("Tokenizing...\n")
tokenized_texts, word2idx, max_len = tokenize(train_node_text["fulltext"])
input_ids = encode(tokenized_texts, word2idx, max_len)

# Load pretrained vectors
embeddings = load_pretrained_vectors(word2idx, "fastText/crawl-300d-2M.vec")
embeddings = torch.tensor(embeddings)

Tokenizing...

Loading pretrained vectors...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



There are 52399 / 117875 pretrained vectors found.


In [ ]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)

def data_loader(train_inputs, val_inputs, train_labels, val_labels,
                batch_size):
  
    # Convert data type to torch.Tensor
    train_inputs, val_inputs, train_labels, val_labels =\
    tuple(torch.tensor(data) for data in
          [train_inputs, val_inputs, train_labels, val_labels])

    # Specify batch_size
    batch_size = batch_size

    # Create DataLoader for training data
    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = torch.utils.data.DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

In [ ]:
from sklearn.model_selection import train_test_split

labels = np.array(train_node_text["label"])

# Train Valid Split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, labels, test_size=0.2, random_state=42)
# Load data to PyTorch DataLoader
train_dataloader, valid_dataloader = data_loader(train_inputs, val_inputs, train_labels, val_labels, 
                                                 batch_size=64)

### CNN-Static/non static


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_NLP(nn.Module):
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[3, 4,5],
                 num_filters=[100, 100,100],
                 num_classes=20,
                 dropout=0.5):

        super(CNN_NLP, self).__init__()
        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
        
        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        
        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(self.dropout(x_fc))
        return logits

In [ ]:
import torch.optim as optim

def initilize_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    filter_sizes=[3,4,5],
                    num_filters=[100,100,100],
                    num_classes=20,
                    dropout=0.5,
                    learning_rate=1e-1):

    # Instantiate CNN model
    cnn_model = CNN_NLP(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        filter_sizes=filter_sizes,
                        num_filters=num_filters,
                        num_classes=num_classes,
                        dropout=0.5)
    
    # Send model to `device` (GPU/CPU)
    cnn_model.to(device)
    optimizer = optim.Adadelta(cnn_model.parameters(),
                                lr=learning_rate,
                                rho=0.95)

    return cnn_model, optimizer

### Train and evaluate


In [ ]:
def train(model, dataloader, optimizer, criterion):
    total_loss, total_correct, total_prediction = 0.0, 0.0, 0.0
    model.train()
    for X, y in dataloader:
        logits = model(X.cuda())
        predictions = torch.max(logits, dim=-1)[1]
        loss = criterion(logits, y.cuda())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_correct += torch.eq(predictions, y.cuda()).sum().item()
        total_prediction += y.size(0)
    return total_loss / len(dataloader), total_correct / total_prediction

# evaluate a model
# DO NOT MODIFY
def evaluate(model, dataloader, criterion):  
    total_loss, total_correct, total_prediction = 0.0, 0.0, 0.0
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            logits = model(X.cuda())
            predictions = torch.max(logits, dim=-1)[1]
            loss = criterion(logits, y.cuda())

            total_loss += loss.item()
            total_correct += torch.eq(predictions, y.cuda()).sum().item()
            total_prediction += y.size(0)
    return total_loss / len(dataloader), total_correct / total_prediction

In [ ]:
# This result just an example to show to code works. Original output and hyperameter tuninig is in other notebook 
# and take long time to run. so I didnot show it here.
EPOCHS=10
criterion= nn.CrossEntropyLoss()
#set_seed(42)
cnn_non_static, optimizer = initilize_model(pretrained_embedding=embeddings,
                                        freeze_embedding=False,# false is to cnn_non_static/ True is cnn_static
                                        learning_rate=2,
                                        dropout=0.5)
best_valid_acc = 0.0
best_state_dict = copy.deepcopy(cnn_non_static.state_dict())
for epoch in range(EPOCHS):
    train_loss, train_acc = train(cnn_non_static, train_dataloader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(cnn_non_static, valid_dataloader, criterion)

    print('Epoch {} | Train loss {:.3f} | Valid loss {:.3f} | Valid acc {:.3f}'.format(epoch, train_loss, valid_loss, valid_acc))


    if valid_acc > best_valid_acc:
          best_valid_acc = valid_acc
          best_state_dict = copy.deepcopy(cnn_non_static.state_dict())
          es = 0
          #torch.save(net.state_dict(), "model_" + str(fold) + 'weight.pt')
    # early stopping      
    else:
          es += 1
          print("Counter {} of 5".format(es))

          if es > 4:
              print("Early stopping with best_acc: ", best_valid_acc, "and val_acc for this epoch: ", valid_acc, "...")
              break


Epoch 0 | Train loss 0.977 | Valid loss 0.694 | Valid acc 0.793
Epoch 1 | Train loss 0.761 | Valid loss 0.687 | Valid acc 0.799
Epoch 2 | Train loss 0.678 | Valid loss 0.680 | Valid acc 0.808
Epoch 3 | Train loss 0.609 | Valid loss 0.709 | Valid acc 0.806
Counter 1 of 5
Epoch 4 | Train loss 0.537 | Valid loss 0.757 | Valid acc 0.804
Counter 2 of 5
Epoch 5 | Train loss 0.480 | Valid loss 0.751 | Valid acc 0.805
Counter 3 of 5
Epoch 6 | Train loss 0.436 | Valid loss 0.787 | Valid acc 0.808
Epoch 7 | Train loss 0.389 | Valid loss 0.790 | Valid acc 0.809
Epoch 8 | Train loss 0.350 | Valid loss 0.867 | Valid acc 0.809
Epoch 9 | Train loss 0.322 | Valid loss 0.920 | Valid acc 0.811


### Prediciton

In [ ]:
def predict(text, model=cnn_static.to("cpu"), max_len=1000):

    # Tokenize, pad and encode text
    tokens = word_tokenize(text)
    padded_tokens = tokens + ['<pad>'] * (max_len - len(tokens))
    input_id = [word2idx.get(token, word2idx['<unk>']) for token in padded_tokens]

    # Convert to PyTorch tensors
    input_id = torch.tensor(input_id).unsqueeze(dim=0)

    # Compute logits
    logits = model.forward(input_id)

    #  Compute probability
    predictions = torch.max(logits, dim=-1)[1]
    return predictions.item()

In [ ]:
# cnn_static=model.load_state_dict(best_state_dict)
target=[]
for i in range(len(test_node_text["fulltext"])): 
  pre=predict(test_node_text["fulltext"][i],cnn_static)
  target.append(pre)

In [ ]:
test_node_text["label"]=target
final_result_cnn_non_static=test_node_text.drop(columns=["paper id","title","abstract"])
final_result_cnn_non_static.columns=["id","label"]

In [ ]:
final_result_cnn_non_static.to_csv(r"/content/drive/My Drive/Colab Notebooks/final_result_cnn.csv",index=False)

##Two channels CNN(**no for grading**)


### Preprocess

In [ ]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)
labels = np.array(train_node_text["label"])

In [ ]:
from sklearn.model_selection import train_test_split

## split data into training, validation, and test data (features and labels, x and y)
train_df, val_df= train_test_split(
    train_node_text, test_size=0.2, random_state=42)

In [ ]:
# source : https://gist.github.com/lextoumbourou/8f90313cbc3598ffbabeeaa1741a11c8
# to use DataFrame as a Data source

class DataFrameDataset(data.Dataset):

    def __init__(self, df, fields, is_test=False, **kwargs):
        examples = []
        for i, row in df.iterrows():
            label = row.label if not is_test else None
            text = row.fulltext
            examples.append(data.Example.fromlist([text, label], fields))

        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    @classmethod
    def splits(cls, fields, train_df, val_df=None, **kwargs):
        train_data, val_data = (None, None)
        data_field = fields

        if train_df is not None:
            train_data = cls(train_df.copy(), data_field, **kwargs)
        if val_df is not None:
            val_data = cls(val_df.copy(), data_field, **kwargs)

        return tuple(d for d in (train_data, val_data) if d is not None)

In [ ]:
fields = [('text',TEXT), ('label',LABEL)]

train_ds, val_ds = DataFrameDataset.splits(fields, train_df=train_df, val_df=val_df)

In [ ]:
TEXT.build_vocab(train_ds, 
                 vectors = 'glove.840B.300d',
                 unk_init = torch.Tensor.zero_)
word_embeddings = TEXT.vocab.vectors
vocab = TEXT.vocab
LABEL.build_vocab(train_ds)

In [ ]:
BATCH_SIZE = 128
#  BucketIterator : Defines an iterator that batches examples of similar lengths together to minimize the amount of padding needed.
# by setting sort_within_batch = True.
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_ds, val_ds), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

### Multi-channel CNN

In [ ]:
class TextCNN(nn.Module):
    def __init__(self, config, vocab_size, word_embeddings):
        super(TextCNN, self).__init__()
        self.config = config
        
        # Embedding Layer
        self.embeddings = nn.Embedding(vocab_size, self.config.embed_dim)
        self.embeddings.weight = nn.Parameter(word_embeddings, requires_grad=False)
        
        # This stackoverflow thread clarifies how conv1d works
        # https://stackoverflow.com/questions/46503816/keras-conv1d-layer-parameters-filters-and-kernel-size/46504997
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels=self.config.embed_dim, out_channels=self.config.num_channels, 
                      kernel_size=self.config.kernel_size[0]),
            nn.ReLU(),
            nn.MaxPool1d(self.config.max_sen_len - self.config.kernel_size[0]+1)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels=self.config.embed_dim, out_channels=self.config.num_channels, 
                      kernel_size=self.config.kernel_size[1]),
            nn.ReLU(),
            nn.MaxPool1d(self.config.max_sen_len - self.config.kernel_size[1]+1)
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels=self.config.embed_dim, out_channels=self.config.num_channels, 
                      kernel_size=self.config.kernel_size[2]),
            nn.ReLU(),
            nn.MaxPool1d(self.config.max_sen_len - self.config.kernel_size[2]+1)
        )
        
        self.dropout = nn.Dropout(self.config.dropout_keep)
        
        # Fully-Connected Layer
        self.fc = nn.Linear(self.config.num_channels*len(self.config.kernel_size), self.config.output_size)

        # Softmax non-linearity
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        x=x.transpose(0,1)
        # x.shape = (max_sen_len, batch_size)
        embedded_sent = self.embeddings(x).permute(1,2,0)
        # embedded_sent.shape = (batch_size=64,embed_size=300,max_sen_len=1200)
        
        conv_out1 = self.conv1(embedded_sent).squeeze(2) #shape=(64, num_channels, 1) (squeeze 1)
        conv_out2 = self.conv2(embedded_sent).squeeze(2)
        conv_out3 = self.conv3(embedded_sent).squeeze(2)
        print(conv_out3.shape)

        out = torch.cat((conv_out1, conv_out2, conv_out3), 1)
        out = F.dropout(out)
        out = self.fc(out)
        
        return out

    def add_optimizer(self, optimizer):
        self.optimizer = optimizer
    
    def reduce_lr(self):
        print("Reducing LR")
        for g in self.optimizer.param_groups:
            g['lr'] = g['lr'] / 2
                
    def run_epoch(self, train_iterator, val_iterator, epoch):
        train_losses = []
        val_accuracies = []
        losses = []
        
        # Reduce learning rate as number of epochs increase
        if (epoch == int(self.config.max_epochs/3)) or (epoch == int(2*self.config.max_epochs/3)):
            self.reduce_lr()
            
        for i, batch in enumerate(train_iterator):
            
            self.optimizer.zero_grad()
            x =batch[0].long().cuda()
            y = batch[1].long().cuda()
            y_pred = model(x)
            # y_pred = model(x)
            # print(f'y_pred',y_pred.shape)
            loss = criterion(y_pred, y)
            loss.backward()
            losses.append(loss.cpu().data.numpy())
            self.optimizer.step()
    
            if i % 1000 == 0:
                # print("Iter: {}".format(i+1))
                avg_train_loss = np.mean(losses)
                train_losses.append(avg_train_loss)
                # print("\tAverage training loss: {:.5f}".format(avg_train_loss))
                losses = []
                
                # Evalute Accuracy on validation set
                val_accuracy = evaluate_model(self, val_iterator)
                print("\tVal Accuracy: {:.4f}".format(val_accuracy))
                self.train()
                
        return train_losses, val_accuracies

### Train and evaluate


In [ ]:
def accuracy(probs, target):
  winners = probs.argmax(dim=1)
  corrects = (winners == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy

In [ ]:
from sklearn.metrics import accuracy_score
def evaluate_model(model, iterator):
    for idx,batch in enumerate(iterator):
        x =batch[0].long().cuda()
        y = batch[1].long().cuda()
        y_pred = model(x)
        acc = accuracy(y_pred, y)
    return acc

In [ ]:
model = TextCNN(config, len(vocab),word_embeddings)
model.to(device)
model.train()
optimizer = optim.SGD(model.parameters(), lr=config.lr)
criterion = nn.CrossEntropyLoss()
model.add_optimizer(optimizer)
model.add_loss_op(criterion)

In [ ]:
train_losses = []
val_accuracies = []

for i in range(config.max_epochs):
    print ("Epoch: {}".format(i))
    train_loss,val_accuracy = model.run_epoch(train_iterator, valid_iterator, i)
    train_losses.append(train_loss)
    val_accuracies.append(val_accuracy)

train_acc = evaluate_model(model, train_iterator)
val_acc = evaluate_model(model, val_iterator)

print ('Final Training Accuracy: {:.4f}'.format(train_acc))
print ('Final Validation Accuracy: {:.4f}'.format(val_acc))


## Bidirectional LSTM-(**no for grading**)

### LSTM Model

In [ ]:
class LSTM_net(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, 
                                      padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        
        self.fc2 = nn.Linear(hidden_dim, 20)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        # text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        # embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu())
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        # output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        # output = [sent len, batch size, hid dim * num directions]
        # output over padding tokens are zero tensors
        
        # hidden = [num layers * num directions, batch size, hid dim]
        # cell = [num layers * num directions, batch size, hid dim]
        
        # concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        # and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        hidden = self.relu(hidden)
        output = self.fc1(hidden)
        output = self.dropout(self.fc2(output)) #hidden = [batch size, hid dim * num directions]
            
        return output

In [ ]:
# Hyperparameters
learning_rate = 0.005
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 20
N_LAYERS = 5
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # padding

In [ ]:
model = LSTM_net(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,PAD_IDX)

In [ ]:
pretrained_embeddings = embeddings
print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)

In [ ]:
#  to initiaise padded to zeros
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
print(model.embedding.weight.data)

### Train and evaluate


In [ ]:
model.to(device) #CNN to GPU
# Loss and optimizer
criterion =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train(model, iterator):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        text, text_lengths = batch.text
        
        optimizer.zero_grad()
        predictions = model(text, text_lengths)
        loss = criterion(predictions, batch.label.long().squeeze())
        acc = accuracy(predictions, batch.label)

        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()

    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label.long().squeeze())
            acc = accuracy(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator),epoch_acc / len(iterator)

## XLNet

### Load package

In [ ]:
!pip install -q -U watermark

In [ ]:
!pip install SentencePiece

In [ ]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.7.10
IPython version      : 5.5.0

numpy       : 1.19.5
pandas      : 1.1.5
torch       : 1.8.1+cu101
transformers: 4.4.2



In [ ]:
from transformers import XLNetTokenizer, XLNetModel
PRE_TRAINED_MODEL_NAME = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
input_txt="a"
encodings = tokenizer.encode_plus(input_txt, add_special_tokens=True,truncation=True, max_length=1000, return_tensors='pt', return_token_type_ids=False, return_attention_mask=True, pad_to_max_length=False)

In [ ]:
token_lens = []

for txt in train_node_text["fulltext"]:
  tokens = tokenizer.encode(txt, max_length=512)
  token_lens.append(len(tokens))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
class topicDataset(Dataset):

    def __init__(self, fulltext, label, tokenizer, max_len):
        self.fulltext = fulltext
        self.label = label
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.fulltext)
    
    def __getitem__(self, item):
        fulltext = str(self.fulltext[item])
        target = self.label[item]

        encoding = self.tokenizer.encode_plus(
        fulltext,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=False,
        return_attention_mask=True,
        return_tensors='pt',
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'text': fulltext,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = topicDataset(
    fulltext=df.fulltext.to_numpy(),
    label=df.label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=2
  )

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(train_node_text, test_size=0.2, random_state=101)
df_test=test_node_text

In [ ]:
BATCH_SIZE = 4
MAX_LEN=512
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
from transformers import XLNetForSequenceClassification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 20)
model = model.to(device)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

EPOCHS = 1

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

### Train and Evaluate

In [ ]:
from sklearn import metrics
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    acc = 0
    counter = 0
  
    for d in data_loader:
        input_ids = d["input_ids"].reshape(4,512).to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
        loss = outputs[0]
        logits = outputs[1]

        # preds = preds.cpu().detach().numpy()
        _, prediction = torch.max(outputs[1], dim=1)
        targets = targets.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, prediction)

        acc += accuracy
        losses.append(loss.item())
        
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        counter = counter + 1

    return acc / counter, np.mean(losses)

In [ ]:
from collections import defaultdict
from keras.preprocessing.sequence import pad_sequences

In [ ]:
#show as an example for training: stop running given long time to run
%%time
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,     
        optimizer, 
        device, 
        scheduler, 
        len(df_train)
    )

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader, 
        device, 
        len(df_val)
    )

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/xlnet_model.bin')
        best_accuracy = val_acc

### Prediction

In [ ]:
#load model save before
model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/xlnet_model2.bin"))

<All keys matched successfully>

In [ ]:
def predict(text):
    text = text

    encoded_text= tokenizer.encode_plus(
    text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=False,
    return_attention_mask=True,
    return_tensors='pt',
    )

    input_ids = pad_sequences(encoded_text['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
    input_ids = input_ids.astype(dtype = 'int64')
    input_ids = torch.tensor(input_ids) 

    attention_mask = pad_sequences(encoded_text['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
    attention_mask = attention_mask.astype(dtype = 'int64')
    attention_mask = torch.tensor(attention_mask) 

    input_ids = input_ids.reshape(1,512).to(device)
    attention_mask = attention_mask.to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    outputs = outputs[0][0].cpu().detach()

    probs = F.softmax(outputs, dim=-1).cpu().detach().numpy().tolist()
    _, prediction = torch.max(outputs, dim =-1)
    return prediction

In [ ]:
target=[]
for i in range(len(df_test["fulltext"])): 
  pre=predict(df_test["fulltext"][i])
  target.append(pre.item())
  #print(target)

AttributeError: ignored

In [ ]:
df_test["label"]=target
final_result_XL=df_test
final_result_XL.columns=["id","label"]

In [ ]:
final_result_XL.to_csv(r"/content/drive/My Drive/Colab Notebooks/final_result_XL.csv",index=False)

## RoBERta-(**no for grading**)

### Preprocess

In [ ]:
from pytorch_transformers import RobertaModel, RobertaTokenizer
from pytorch_transformers import RobertaForSequenceClassification, RobertaConfig
from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig

roberta_tok = RobertaTokenizer.from_pretrained('roberta-base')

100%|██████████| 456318/456318 [00:00<00:00, 1287039.88B/s]


In [ ]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=False,
    seed = 2019,
    roberta_model_name='roberta-base', # can also be exchnaged with roberta-large 
    max_lr=1e-6,
    epochs=1,
    use_fp16=False,
    bs=64, 
    max_seq_len=256, 
    num_labels = 20,
    hidden_dropout_prob=.05,
    hidden_size=768, # 1024 for roberta-large
    start_tok = "<s>",
    end_tok = "</s>",
)

In [ ]:
from fastai.text import *
from fastai.metrics import *
class FastAiRobertaTokenizer(BaseTokenizer):
    def __init__(self, tokenizer: RobertaTokenizer, max_seq_len: int=128, **kwargs): 
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len 
    def __call__(self, *args, **kwargs): 
        return self 
    def tokenizer(self, t:str) -> List[str]: 
        return ["<s>"] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["</s>"]

In [ ]:
roberta_tok = RobertaTokenizer.from_pretrained("roberta-large")

fastai_tokenizer = Tokenizer(tok_func=FastAiRobertaTokenizer(roberta_tok, max_seq_len=config.max_seq_len), 
                             pre_rules=[], post_rules=[])

In [ ]:
# create fastai vocabulary for roberta
path = Path()
roberta_tok.save_vocabulary(path)

with open('vocab.json', 'r') as f:
    roberta_vocab_dict = json.load(f)
    
fastai_roberta_vocab = Vocab(list(roberta_vocab_dict.keys()))

### RoBERta Model

In [ ]:
pip install pytorch_transformers

In [ ]:
class RobertaTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
         super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False)

class RobertaNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


def get_roberta_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for Roberta
    We remove sos and eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original Roberta model.
    """
    return [RobertaTokenizeProcessor(tokenizer=tokenizer), RobertaNumericalizeProcessor(vocab=vocab)]

In [ ]:
class RobertaDataBunch(TextDataBunch):
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs:int=64, val_bs:int=None, pad_idx=1,
               pad_first=True, device:torch.device=None, no_check:bool=False, backwards:bool=False, 
               dl_tfms:Optional[Collection[Callable]]=None, **dl_kwargs) -> DataBunch:
        "Function that transform the `datasets` in a `DataBunch` for classification. Passes `**dl_kwargs` on to `DataLoader()`"
        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
        val_bs = ifnone(val_bs, bs)
        collate_fn = partial(pad_collate, pad_idx=pad_idx, pad_first=pad_first, backwards=backwards)
        train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs)
        train_dl = DataLoader(datasets[0], batch_size=bs, sampler=train_sampler, drop_last=True, **dl_kwargs)
        dataloaders = [train_dl]
        for ds in datasets[1:]:
            lengths = [len(t) for t in ds.x.items]
            sampler = SortSampler(ds.x, key=lengths.__getitem__)
            dataloaders.append(DataLoader(ds, batch_size=val_bs, sampler=sampler, **dl_kwargs))
        return cls(*dataloaders, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)

In [ ]:
class RobertaTextList(TextList):
    _bunch = RobertaDataBunch
    _label_cls = TextList

In [ ]:
feat_cols = "fulltext"
label_cols = "label"

In [ ]:
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)
data_R = RobertaTextList.from_df(train_node_text, "/content/drive/My Drive/Colab Notebooks/trainfulltext.csv", cols=feat_cols, processor=processor) \
    .split_by_rand_pct(0.2) \
    .label_from_df(cols=label_cols,label_cls=CategoryList) \
    .add_test(RobertaTextList.from_df(test_node_text, "/content/drive/My Drive/Colab Notebooks/testfulltext.csv", cols=feat_cols, processor=processor)) \
    .databunch(bs=32, pad_first=False, pad_idx=0)

In [ ]:
class CustomRobertatModel(nn.Module):
    def __init__(self,num_labels=20):
        super(CustomRobertatModel,self).__init__()
        self.num_labels = num_labels
        self.roberta = RobertaModel.from_pretrained("roberta-base")
        self.dropout = nn.Dropout(.5)
        self.classifier = nn.Linear(768, num_labels)
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _ , pooled_output = self.roberta(input_ids, token_type_ids, attention_mask)
        logits = self.classifier(pooled_output)        
        return logits

In [ ]:
roberta_model = CustomRobertatModel(num_labels=config.num_labels)

100%|██████████| 501200538/501200538 [00:16<00:00, 30019687.01B/s]


In [ ]:
learner = Learner(data_R, roberta_model, metrics=[accuracy])

NameError: ignored

In [ ]:
def load_model(learner, file_name):
    st = torch.load(file_name)
    learner.model.load_state_dict(st)

In [ ]:
def save_model(learner, file_name):
    st = learner.model.state_dict()
    torch.save(st, file_name)

In [ ]:
#find the best learning rate
# learner.model_dir='/content/drive/MyDrive'
# learner.lr_find()
# learner.recorder.plot()

In [ ]:
learner.model.roberta.train() # setting roberta to train as it is in eval mode by default
learner.fit_one_cycle(15, 1e-5)

NameError: ignored

### Prediction

In [ ]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    """
    the get_preds method does not yield the elements in order by default
    we borrow the code from the RNNLearner to resort the elements into their correct order
    """
    preds = learner.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    return preds[reverse_sampler, :]

test_preds = get_preds_as_nparray(DatasetType.Test)

In [ ]:
sample_submission = pd.read_csv("/content/drive/My Drive/Colab Notebooks/test_clean_text.csv")
sample_submission['label'] = np.argmax(test_preds,axis=1)
sample_submission=sample_submission.drop(columns=["paper id","title","abstract"])
sample_submission.columns=["id","label"]
sample_submission.to_csv("result_reberta_try2_HW2.csv", index=False)

##SVM with TF-IDF 

In [ ]:
x_train=train_node_text["fulltext"]
y_train = train_node_text["label"]

In [ ]:
from sklearn.model_selection import train_test_split
train_x,val_x,train_y,val_y=train_test_split(x_train,y_train,test_size=0.2)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=50000)
tfidf_vect.fit(train_node_text['fulltext'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(val_x)
xtest_tfidf=tfidf_vect.transform(test_node_text.fulltext)

### Train and Evaluate SVM

In [ ]:
# SVM on TF-IDF
from sklearn import svm
clf = svm.SVC()
clf.fit(xtrain_tfidf, train_y)

y_train_pred=clf.predict(xtrain_tfidf)
y_val_pred = clf.predict(xvalid_tfidf)

from sklearn.metrics import accuracy_score
print("Accuracy: {:.2f}%".format(accuracy_score(train_y, y_train_pred) * 100))
print("Accuracy: {:.2f}%".format(accuracy_score(val_y, y_val_pred) * 100))

### Prediction

In [ ]:
y_test_pred = clf.predict(xtest_tfidf)

In [ ]:
test_node_text["label"]=y_test_pred
["label"]=y_test_pred

In [ ]:
test=test_node_text.drop(["paper id","title","abstract"],axis=1)

In [ ]:
test.to_csv("/content/drive/MyDrive/Colab Notebooks/svm.csv")